In [19]:
import pandas as pd
import numpy as np
import torch

In [20]:
data_dir = './datasets/'

In [21]:
original_train_data = pd.read_csv(data_dir + 'train_data.csv')
#augmented_train_data = pd.read_csv(data_dir + 'augmented_data_label_1_pegasus.csv')
augmented_train_data = pd.read_csv(data_dir + 'augmented_data_label_1_parrot.csv')
train_data = pd.concat([original_train_data, augmented_train_data], axis=0)

In [22]:
# create 5 balanced sub-datasets for bagging
num_sub_datasets = 5
train_data_list = []

ones = train_data[train_data['label'] == 1]
zeros = train_data[train_data['label'] == 0]
num = len(ones)

for i in range(num_sub_datasets):
    train_data_list.append(pd.concat([ones, zeros.sample(n=num, random_state=i)], axis=0))

In [23]:
train_data_list[0]['label'].value_counts()

1    2099
0    2099
Name: label, dtype: int64

In [24]:
'''
TODO
1. train the same model on 5 sub-datasets.
2. predict labels using the 5 models (voting).
3. calculate the accuracy and f1 score of the voting result.
'''

'\nTODO\n1. train the same model on 5 sub-datasets.\n2. predict labels using the 5 models (voting).\n3. calculate the accuracy and f1 score of the voting result.\n'